In [1]:
# Import all libraries

import pandas as pd
import urllib as ulib
import requests
import os
import re #Regular Expression
from io import BytesIO

# To parse the document, BeautifulSoup library is used. 
from bs4 import BeautifulSoup

# To read and open zipfile, zipfile module is used
import zipfile
from zipfile import ZipFile
import glob

#####S3#######
import boto
import boto.s3
import sys
from boto.s3.key import Key
##############
######chethan#######
import datetime
import time
ts = time.time()
global dt
global year
dt = datetime.datetime.fromtimestamp(ts).strftime('%Y%m%d_%H%M%S')
#######year for file??????????
#year = '2017'
####################

In [49]:
print("Enter the Year you want the analysis for")
year = input("Enter year from 2003 to 2017: ")


Enter the Year you want the analysis for
Enter year from 2003 to 2017: 2017


In [50]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fh = logging.FileHandler('DATACLEAN_'+year+'_'+dt+'.txt')
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
logger.addHandler(fh)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)
logger.addHandler(ch)

global changeindex 
global nuberofvalidentries
global violationcount
changeindex = [] ##change
logger.debug('Clearing the index array :')
logger.debug(changeindex)

############################################################################################################

2018-10-12 17:43:40,701 - DEBUG - Clearing the index array :
2018-10-12 17:43:40,701 - DEBUG - Clearing the index array :
2018-10-12 17:43:40,701 - DEBUG - Clearing the index array :
2018-10-12 17:43:40,701 - DEBUG - Clearing the index array :
2018-10-12 17:43:40,701 - DEBUG - Clearing the index array :
2018-10-12 17:43:40,708 - DEBUG - []
2018-10-12 17:43:40,708 - DEBUG - []
2018-10-12 17:43:40,708 - DEBUG - []
2018-10-12 17:43:40,708 - DEBUG - []
2018-10-12 17:43:40,708 - DEBUG - []


In [3]:
#Now we need to specify the link of the website from where we want to scrape the data
page=requests.get("https://www.sec.gov/dera/data/edgar-log-file-data-set.html")

# Now that the document has been successfully downloaded, we need to parse the document so that the data needed can be extracted
#An instance of the library needs to be created for parsing
soup=BeautifulSoup(page.content,'html.parser')

In [4]:
# Now we need to define a function which will get us all the year links in a list
#The div_tag_list will give us the div tags which has all the year links
#The a_tag_list will basically store the <a> tags in a list. 
#But this variable contains a list which has a list of all the a_list_tags. So basically the list has just one element.
#So we need to just take that one element as a separate list (named a_tag_list again) and use that list 
def generate_link_list():
    div_tag_list=soup.find_all('div',attrs={'id':'asyncAccordion'})
    a_tag_list=[]
    for div_tag in div_tag_list:
        a_tag=div_tag.find_all('a')
        a_tag_list.append(a_tag)
    a_tag_list=a_tag_list[0]
    year_links=[]
    for aa in a_tag_list:
        link=aa.get('href')
        final_link='https://www.sec.gov'+link
        year_links.append(final_link) 
    return(year_links)


In [5]:

# Now that the links are generated, we need a function to check whether the year input by the user is valid or not.
# For that we will generate a list which has all the valid years and then will define a function which will check the userinput
    
def valid_year(user_year_input):
    valid_year_list=[]
    for i in range(2003,2018):
        valid_year_list.append(i)
    if user_year_input in valid_year_list:
        return('The year is valid')
    else:
        return('The year is invalid')
        

In [6]:

# Now we need to define a function which will select the appropriate year link depending upon the user input 

def user_input_year_link(year):
    link_list=generate_link_list()
    for year_link in link_list:
        if year in year_link:
            year=year_link
    zip_files = ulib.request.urlopen(year)
    soup=BeautifulSoup(zip_files,'html.parser')
    a_tag=soup.find_all('a')
    a_tag_list=[]
    for i in a_tag:
        links=i.get('href')
        a_tag_list.append(links)
    first_day_zip_links=[]
    for i in a_tag_list:
        if '01.zip' in i:
            first_day_zip_links.append(i)
    return(first_day_zip_links)
    


In [7]:
# Now we need to create a file path on our local where all the zip files will be extracted and csv will be stored

def zip_csv_on_local(year):
    folder=str(year)
    #path= str(os.getcwd())+'/'+folder+dt
    path= str(os.getcwd())+'/'+folder
    os.makedirs(path)
    a=user_input_year_link(year)
    for i in a:
        with ulib.request.urlopen(i) as firstMonth:
            with ZipFile(BytesIO(firstMonth.read())) as unzippedFile:
                unzippedFile.extractall(path)


In [48]:
def createfinalfolder(year):
    ts = time.time()
    dt = datetime.datetime.fromtimestamp(ts).strftime('%Y%m%d_%H%M%S')
    folder=str(year)+str(dt)
    path= str(os.getcwd())+'/'+folder
    os.makedirs(path)
    return path
    

In [9]:


# Now we need to load data from each of the csv files that are in the folder to separate dataframes

def create_dataframes(i,path):
    file=glob.glob(path+'/log*.csv')
   # def read_csv(list_val):
    #    return pd.read_csv(list_val)
    val = file[i]
    df=pd.read_csv(val,low_memory=False)
    return df


In [10]:
##########################################chethan##############################
#The null report
def tofindnull(dt):
    logger.info("Finding the null values")
    null_columns=dt.columns[dt.isnull().any()]
    logger.info("The summary of null values : ")
    logger.debug(dt[null_columns].isnull().sum())


In [11]:
#To replace the null values with 0
def cleannull(cdt):
    logger.info("Replacing all the null values with zero")
    cdt.fillna(0, inplace=True)
    logger.info("NaN values are replaced successfuly")
    logger.info("The null report after cleaning")
    tofindnull(cdt)
    return cdt


In [12]:
def datacorrection(nuberofvalidentries,column,index,dfclean):
    logger.info("The data correction is being done on "+str(column))
    for i in index:
        dfclean.loc[index, column] = 0
        logger.debug("The value after changing : ")
        logger.debug(dfclean.iloc[i][column])
    logger.info("nuber of valid entries : "+str(nuberofvalidentries))
    logger.info("number of non zero values"+str(dfclean[column].astype(bool).sum(axis=0)))
    if(nuberofvalidentries == dfclean[column].astype(bool).sum(axis=0)):
        logger.info("The dataclean was successful for "+str(column))
        return dfclean
    else:
        logger.info("The dataclean was not successful for "+str(column))
        return dfclean 


In [13]:
#To check the anamolies at "browser" column
def browserclean(cdr):  

    totalrows = len(cdr)
    logger.info("The total entries in the dataframe : "+str(totalrows))
    #The expected values
    browsers = ["mie","fox","saf","chr","sea","opr","oth","mac","lin","iph","ipd","and","rim","iem"]
    logger.info("The valid browsers name : ")
    logger.info(browsers)
    logger.info("Finding whether browser analysis is possible")
    nuberofvalidentries = 0
    violationcount = 0
    i=0
    for x in cdr["browser"]:
          ##change
        violation = 1
        for y in browsers:
            if (x == y):
                #logger.info("The correct value : ",x)
                violation = 0
                break
        if (violation == 1):
            #logger.info("the bad value is",x)
            if(x != 0):
                changeindex.append(i) ##change
            violationcount = violationcount + 1
            #x = "NotAllowed"
        i = i+1  ##change
    logger.info("The index at which values need to be changed : ")
    logger.info(changeindex)
    logger.info("The number of violations : "+str(violationcount))
    logger.info("The total rows : "+str(totalrows))
    nuberofvalidentries = totalrows - violationcount
    logger.info("Total number of valid entries : "+str(nuberofvalidentries))
    percentofvalidvalues = (nuberofvalidentries/totalrows)*100
    if (percentofvalidvalues >= 5):
        logger.info("The browser analysis is possible as the value is : "+str(percentofvalidvalues))
        return 1,nuberofvalidentries
    else:
        logger.info("The browser analysis is not possible as the value is : "+str(percentofvalidvalues))
        return 0,nuberofvalidentries	


In [14]:
def findclean(cdr):    
    totalrows = len(cdr)
    logger.info("The total entries in the dataframe : "+str(totalrows))
    finds = ["0.0","1.0","2.0","3.0","4.0","5.0","6.0","7.0","8.0","9.0","10.0"]
    logger.info("The valid find values : ")
    logger.info(finds)
    logger.info("Finding whether find analysis is possible")
    nuberofvalidentries = 0
    i=0
    violationcount = 0
    for x in cdr["find"]:
        violation = 1
        for y in finds:
            if (str(x) == y):
                #logger.info("The correct value : ",x)
                violation = 0
                break
        if (violation == 1):
            logger.debug("the bad value is"+str(x))
            if(x != 0):
                changeindex.append(i) ##change
            violationcount = violationcount + 1
        i = i+1
    logger.info("The index at which values need to be changed : ")
    logger.debug(changeindex)
    logger.info("The number of violations : "+str(violationcount))
    logger.info("The total rows : "+str(totalrows))
    nuberofvalidentries = totalrows - violationcount
    logger.info("Total number of valid entries : "+str(nuberofvalidentries))
    percentofvalidvalues = (nuberofvalidentries/totalrows)*100
    if (percentofvalidvalues >= 5):
        logger.info("The find analysis is possible as the value is : "+str(percentofvalidvalues))
        return 1,nuberofvalidentries
    else:
        logger.info("The find analysis is not possible as the value is : "+str(percentofvalidvalues))
        return 0,nuberofvalidentries


In [15]:
def allfloatclean(column,cdr):   
    logger.info("cleaning the column : "+str(column))
    totalrows = len(cdr)
    logger.info("The total entries in the dataframe : "+str(totalrows))
    floats = ["0.0","1.0"]
    logger.info("The valid values : ")
    logger.info(floats)
    logger.info("Finding whether analysis is possible")
    nuberofvalidentries = 0
    i=0
    violationcount = 0
    for x in cdr[column]:
        violation = 1
        for y in floats:
            if (str(x) == y):
                #logger.info("The correct value : ",x)
                violation = 0
                break
        if (violation == 1):
            if(x != 0):
                changeindex.append(i) ##change
            logger.debug("the bad value is"+str(x))
            violationcount = violationcount + 1
        i = i+1
    logger.debug("The index at which values need to be changed : ")
    logger.debug(changeindex)
    logger.info("The number of violations : "+str(violationcount))
    logger.info("The total rows : "+str(totalrows))
    nuberofvalidentries = totalrows - violationcount
    logger.info("Total number of valid entries : "+str(nuberofvalidentries))
    percentofvalidvalues = (nuberofvalidentries/totalrows)*100
    if (percentofvalidvalues >= 5):
        logger.info("The analysis is possible as the value is : "+str(percentofvalidvalues))
        return 1,nuberofvalidentries
    else:
        logger.info("The analysis is not possible as the value is : "+str(percentofvalidvalues))
        return 0,nuberofvalidentries


In [16]:
def floattypecheck(column,dfs):
    logger.info("cleaning the column : "+str(column))
    totalrows = len(dfs)
    logger.info("The total entries in the dataframe : "+str(totalrows))
    logger.info("The valid values : any valid number of type float")
    logger.info("Finding whether analysis is possible")
    nuberofvalidentries = 0
    i=0
    violationcount = 0
    for x in dfs[column]:
        violation = 1
        if(type(x) == float):
            #logger.info("float")
            violation = 0
        else:
            logger.info("not float")
        if (violation == 1):
            logger.debug("the bad value is"+str(x))
            if(x != 0):
                changeindex.append(i) ##change
            violationcount = violationcount + 1
        i = i+1
    logger.info("The index at which values need to be changed : ")
    logger.info(changeindex)
    logger.info("The number of violations : "+str(violationcount))
    logger.info("The total rows : "+str(totalrows))
    nuberofvalidentries = totalrows - violationcount
    logger.info("Total number of valid entries : "+str(nuberofvalidentries))
    percentofvalidvalues = (nuberofvalidentries/totalrows)*100
    if (percentofvalidvalues >= 5):
        logger.info("The analysis is possible as the value is : "+str(percentofvalidvalues))
        return 1,nuberofvalidentries
    else:
        logger.info("The analysis is not possible as the value is : "+str(percentofvalidvalues))
        return 0,nuberofvalidentries


In [17]:
def codecheck(column,dfs):
    logger.info("cleaning the column : "+str(column))
    totalrows = len(dfs)
    logger.info("The total entries in the dataframe : "+str(totalrows))
    logger.info("The valid values : 1xx 2xx 3xx 4xx 5xx")
    logger.info("Finding whether analysis is possible")
    validcodes = ["1","2","3","4","5"]
    nuberofvalidentries = 0
    i=0
    violationcount = 0
    for x in dfs[column]:
        violation = 1
        for y in validcodes:
            if (str(int(x/100.0)) == y):
                #logger.info("The correct value : ",x)
                violation = 0
                break
        if (violation == 1):
            #logger.info("the bad value is",x)
            violationcount = violationcount + 1
            if(x != 0):
                changeindex.append(i) ##change
        i = i+1
    logger.info("The index at which values need to be changed : ")
    logger.info(changeindex)
    logger.info("The number of violations : "+str(violationcount))
    logger.info("The total rows : "+str(totalrows))
    nuberofvalidentries = totalrows - violationcount
    logger.info("Total number of valid entries : "+str(nuberofvalidentries))
    percentofvalidvalues = (nuberofvalidentries/totalrows)*100
    if (percentofvalidvalues >= 5):
        logger.info("The analysis is possible as the value is : "+str(percentofvalidvalues))
        return 1,nuberofvalidentries
    else:
        logger.info("The analysis is not possible as the value is : "+str(percentofvalidvalues))
        return 0,nuberofvalidentries


In [18]:
def dateclean(column,dfs):
    logger.info("cleaning the column : "+str(column))
    totalrows = len(dfs)
    logger.info("The total entries in the dataframe : "+str(totalrows))
    logger.info("The valid values : year-month-day")
    logger.info("Finding whether analysis is possible")
    nuberofvalidentries = 0
    i=0
    violationcount = 0
    for x in dfs[column]:
        violation = 1
        inputDate = str(x)
        #logger.info(inputDate)
        year,day,month = inputDate.split('-')
        isValidDate = True
        try :
            datetime.datetime(int(year),int(month),int(day))
        except ValueError :
            isValidDate = False
        if(isValidDate) :
            #logger.info ("Input date is valid ..")
            violation = 0
        if (violation == 1):

            logger.debug("the bad value is"+str(x))
            if(x != 0):
                changeindex.append(i) ##change
            violationcount = violationcount + 1
        i = i+1
    logger.info("The index at which values need to be changed : ")
    logger.info(changeindex)
    logger.info("The number of violations : "+str(violationcount))
    logger.info("The total rows : "+str(totalrows))
    nuberofvalidentries = totalrows - violationcount
    logger.info("Total number of valid entries : "+str(nuberofvalidentries))
    percentofvalidvalues = (nuberofvalidentries/totalrows)*100
    if (percentofvalidvalues >= 5):
        logger.info("The analysis is possible as the value is : "+str(percentofvalidvalues))
        return 1,nuberofvalidentries
    else:
        logger.info("The analysis is not possible as the value is : "+str(percentofvalidvalues))
        return 0,nuberofvalidentries


In [19]:
def timeclean(column,dfs):
    logger.info("cleaning the column : "+str(column))
    totalrows = len(dfs)
    logger.info("The total entries in the dataframe : "+str(totalrows))
    logger.info("The valid values : %H:%M:%S")
    logger.info("Finding whether analysis is possible")
    nuberofvalidentries = 0
    i=0
    violationcount = 0
    for x in dfs[column]:
        violation = 1
        timeformat = "%H:%M:%S"
        try:
            validtime = datetime.datetime.strptime(x, timeformat)
            #logger.info("valid")
            violation = 0
        except ValueError:
            #Do your logic for invalid format (maybe logger.info some message?).
            logger.debug(" not valid")
        if (violation == 1):
            #logger.info("the bad value is",x)
            if(x != 0):
                changeindex.append(i) ##change
            violationcount = violationcount + 1
        i = i+1
    logger.info("The index at which values need to be changed : ")
    logger.info(changeindex)
    logger.info("The number of violations : "+str(violationcount))
    logger.info("The total rows : "+str(totalrows))
    nuberofvalidentries = totalrows - violationcount
    logger.info("Total number of valid entries : "+str(nuberofvalidentries))
    percentofvalidvalues = (nuberofvalidentries/totalrows)*100
    if (percentofvalidvalues >= 5):
        logger.info("The analysis is possible as the value is : "+str(percentofvalidvalues))
        return 1,nuberofvalidentries
    else:
        logger.info("The analysis is not possible as the value is : "+str(percentofvalidvalues))
        return 0,nuberofvalidentries


In [20]:
def ipclean(column,dfs):
    logger.info("cleaning the column : "+str(column))
    totalrows = len(dfs)
    logger.info("The total entries in the dataframe : "+str(totalrows))
    logger.info("The valid values : ###.###.###.xxx")
    logger.info("Finding whether analysis is possible")
    nuberofvalidentries = 0
    i=0
    violationcount = 0
    for x in dfs[column]:
        violation = 1
        ip = str(x)
        p1,p2,p3,p4 = x.split('.')
        if((type(p1) == str) and (type(p2) == str) and (type(p3) == str) and (type(p4) == str)):
            violation = 0
        if (violation == 1):
            #logger.info("the bad value is",x)
            if(x != 0):
                changeindex.append(i) ##change
            violationcount = violationcount + 1
        i = i+1
    logger.info("The index at which values need to be changed : ")
    logger.info(changeindex)
    logger.info("The number of violations : "+str(violationcount))
    logger.info("The total rows : "+str(totalrows))
    nuberofvalidentries = totalrows - violationcount
    logger.info("Total number of valid entries : "+str(nuberofvalidentries))
    percentofvalidvalues = (nuberofvalidentries/totalrows)*100
    if (percentofvalidvalues >= 5):
        logger.info("The analysis is possible as the value is : "+str(percentofvalidvalues))
        return 1,nuberofvalidentries
    else:
        logger.info("The analysis is not possible as the value is : "+str(percentofvalidvalues))
        return 0,nuberofvalidentries


In [21]:
def tofindnonzero(column,dfnz):
    logger.info("cleaning the column : "+str(column))
    totalrows = len(dfnz)
    logger.info("The total entries in the dataframe : "+str(totalrows))   
    logger.info("Finding whether analysis is possible")
    violationcount = "global"
    violationcount = totalrows - dfnz[column].astype(bool).sum(axis=0)
    logger.info("The number of violations : "+str(violationcount))
    logger.info("The total rows : "+str(totalrows))
    nuberofvalidentries = totalrows - violationcount
    logger.info("Total number of valid entries : "+str(nuberofvalidentries))
    percentofvalidvalues = (nuberofvalidentries/totalrows)*100
    if (percentofvalidvalues >= 10):
        logger.info("The analysis is possible as the value is : "+str(percentofvalidvalues))
        return 1
    else:
        logger.info("The analysis is not possible as the value is : "+str(percentofvalidvalues))
        return 0  


In [22]:
def numberrc(dtrc):
    logger.info("checking whether all columns have same number of rows")
    totalrows = len(dtrc)
    logger.info("The total entries in the dataframe : "+str(totalrows))
    columns = list(dtrc.head(0))
    logger.info("The valid columns"+str(columns))
    i = 0
    for x in columns:
        if (len(dtrc[x]) == totalrows):
            logger.info("No data missing in : "+str(x))
            i = i + 1
        else:
            logger.info("Number of missing data "+str(totalrows-len(dtrc[x])))
    if(i == len(columns)):
        logger.info ("There are no missing data")
        return 1
    else:
        logger.info ("check for missing data")
        return 0      


In [23]:
def dataclean(df1):
    #Null clean is being called
    dfc1 = cleannull(df1)
    logger.info("Null cleaned data")
    #logger.info(dfc1.head(5))
    #############################
    changeindex = []
    logger.info("Checking and cleaning the browser")
    resultbrowser,nv = browserclean(dfc1)
    logger.info("the number of valid values : "+str(nv))
    if (resultbrowser):
        logger.info("The browser data is sufficient for analysis : "+str(dfc1.iloc[10]["date"]))
    else:
        logger.error("The browser data is not sufficient for analysis : "+str(dfc1.iloc[10]["date"]))
        logger.error("The browser data is inconsistent for the date : "+str(dfc1.iloc[10]["date"]))
    #######################################
    logger.info("index of find where wrong values are present :")
    logger.info(changeindex)
    arraysizeforclean = len(changeindex)
    if(arraysizeforclean>0):
        column = "browser"
        logger.info("cleaning the browser : "+str(dfc1.iloc[10]["date"]))
        dfc1 = datacorrection(nv,column,changeindex,dfc1)
    else:
        logger.info("Cleaning of data not required as it doesnt contain wrong values : "+str(dfc1.iloc[10]["date"]))


    logger.info("The (browser)cleaned data ")
    logger.info(dfc1.iloc[10]["date"])
    #dfc1["find"].unique()
    ##Browser clean done####
    ##duhb##
    ######################################################
    changeindex = []
    logger.info("Checking and cleaning the find")
    resultbrowser,nv = findclean(dfc1)
    logger.info("the number of valid values : "+str(nv))
    if (resultbrowser):
        logger.info("The browser data is sufficient for analysis : "+str(dfc1.iloc[10]["date"]))
    else:
        logger.error("The browser data is not sufficient for analysis : "+str(dfc1.iloc[10]["date"]))
        logger.error("The browser data is inconsistent for the date : "+str(dfc1.iloc[10]["date"]))
    ##############################################
    logger.info("index of find where wrong values are present :")
    logger.info(changeindex)
    arraysizeforclean = len(changeindex)
    if(arraysizeforclean>0):
        column = "find"
        logger.info("cleaning the browser : "+str(dfc1.iloc[10]["date"]))
        dfc1 = datacorrection(nv,column,changeindex,dfc1)
    else:
        logger.info("Cleaning of data not required as it doesnt contain wrong values : "+str(dfc1.iloc[10]["date"]))


    logger.info("The (find)cleaned data ")
    logger.info(dfc1.iloc[10]["date"])
    dfc1["find"].unique()
    ##find clean done####
    ###duhb###
    #############################################
    var = ["crawler","idx","norefer","noagent"]
    for x in var:
        logger.info("checking on : "+str(x))
        changeindex = []
        result= 0
        nv = 0
        logger.info("Checking and cleaning the"+str(x))
        result,nv = allfloatclean(x,dfc1)
        logger.info("the number of valid values : "+str(nv))
        if (result):
            logger.info("The "+str(x)+" data is sufficient for analysis : "+str(dfc1.iloc[10]["date"]))
        else:
            logger.error("The "+str(x)+" data is not sufficient for analysis : "+str(dfc1.iloc[10]["date"]))
            logger.error("The "+str(x)+" data is inconsistent for the date : "+str(dfc1.iloc[10]["date"]))
        #calling the cleAN    
        logger.info("index of find where wrong values are present :")
        logger.info(changeindex)
        arraysizeforclean = len(changeindex)
        if(arraysizeforclean>0):
            logger.info("cleaning the"+str(x)+":" +str(dfc1.iloc[10]["date"]))
            dfc1 = datacorrection(nv,x,changeindex,dfc1)
        else:
            logger.info("Cleaning of data not required as it doesnt contain wrong values : "+str(dfc1.iloc[10]["date"]))
    ##dufb
    #############################
    var = ["crawler","idx","norefer","noagent"]
    for x in var:
        logger.info("The float values cleaned data ")
        logger.info(dfc1.iloc[10]["date"])
        logger.info(dfc1[x].unique())
    ################################
    var2 = ["size","zone","cik"]
    for x in var2:
        logger.info("checking on : "+str(x))
        changeindex = []
        result= 0
        nv = 0
        logger.info("Checking and cleaning the"+str(x))
        result,nv = floattypecheck(x,dfc1)
        logger.info("the number of valid values : "+str(nv))
        if (result):
            logger.info("The "+str(x)+" data is sufficient for analysis : "+str(dfc1.iloc[10]["date"]))
        else:
            logger.error("The "+str(x)+" data is not sufficient for analysis : "+str(dfc1.iloc[10]["date"]))
            logger.error("The "+str(x)+" data is inconsistent for the date : "+str(dfc1.iloc[10]["date"]))
        #calling the cleAN    
        logger.info("index of find where wrong values are present :")
        logger.debug(changeindex)
        arraysizeforclean = len(changeindex)
        if(arraysizeforclean>0):
            logger.info("cleaning the"+str(x)+":"+str(dfc1.iloc[10]["date"]))
            dfc1 = datacorrection(nv,x,changeindex,dfc1)
        else:
            logger.info("Cleaning of data not required as it doesnt contain wrong values : "+str(dfc1.iloc[10]["date"]))
    ####duhb####
    ##############################
    var2 = ["size","zone","cik"]
    for x in var2:
        logger.info("The float values cleaned data ")
        logger.info(dfc1.iloc[10]["date"])
        cf = dfc1[x].unique()
        logger.info(cf.dtype)
    ##############################
    ###duhb###
    changeindex = []
    logger.info("Checking and cleaning the code")
    column = "code"
    resultbrowser,nv = codecheck(column,dfc1)
    logger.info("the number of valid values : "+str(nv))
    if (resultbrowser):
        logger.info("The browser data is sufficient for analysis : "+str(dfc1.iloc[10]["date"]))
    else:
        logger.error("The browser data is not sufficient for analysis : "+str(dfc1.iloc[10]["date"]))
        logger.error("The browser data is inconsistent for the date : "+str(dfc1.iloc[10]["date"]))
    ################################################################################################
    logger.info("index of find where wrong values are present :")
    logger.info(changeindex)
    arraysizeforclean = len(changeindex)
    if(arraysizeforclean>0):
        column = "code"
        logger.info("cleaning the code : "+str(dfc1.iloc[10]["date"]))
        dfc1 = datacorrection(nv,column,changeindex,dfc1)
    else:
        logger.info("Cleaning of data not required as it doesnt contain wrong values : "+str(dfc1.iloc[10]["date"]))


    logger.info("The (code)cleaned data ")
    logger.info(dfc1.iloc[10]["date"])
    dfc1["code"].unique()
    ###duhb###
    ##code clean done####
    ###############################
    changeindex = []
    logger.info("Checking and cleaning the date")
    column = "date"
    resultbrowser,nv = dateclean(column,dfc1)
    logger.info("the number of valid values : "+str(nv))
    if (resultbrowser):
        logger.info("The  data is sufficient for analysis : "+str(dfc1.iloc[10]["date"]))
    else:
        logger.error("The data is not sufficient for analysis : "+str(dfc1.iloc[10]["date"]))
        logger.error("The data is inconsistent for the date : "+str(dfc1.iloc[10]["date"]))
    ################################################################################################
    logger.info("index of find where wrong values are present :")
    logger.info(changeindex)
    arraysizeforclean = len(changeindex)
    if(arraysizeforclean>0):
        column = "date"
        logger.info("cleaning the code : "+str(dfc1.iloc[10]["date"]))
        dfc1 = datacorrection(nv,column,changeindex,dfc1)
    else:
        logger.info("Cleaning of data not required as it doesnt contain wrong values : "+str(dfc1.iloc[10]["date"]))


    logger.info("The (date)cleaned data ")
    logger.info(dfc1.iloc[10]["date"])
    logger.debug(dfc1["date"].unique())
    ##duhb####
    ##date clean done####
    ##################################
    changeindex = []
    logger.info("Checking and cleaning the time")
    column = "time"
    resultbrowser,nv = timeclean(column,dfc1)
    logger.info("the number of valid values : "+str(nv))
    if (resultbrowser):
        logger.info("The  data is sufficient for analysis : "+str(dfc1.iloc[10]["date"]))
    else:
        logger.error("The data is not sufficient for analysis : "+str(dfc1.iloc[10]["date"]))
        logger.error("The data is inconsistent for the date : "+str(dfc1.iloc[10]["date"]))
    ################################################################################################
    logger.info("index of find where wrong values are present :")
    logger.info(changeindex)
    arraysizeforclean = len(changeindex)
    if(arraysizeforclean>0):
        column = "time"
        logger.info("cleaning the code : "+str(dfc1.iloc[10]["date"]))
        dfc1 = datacorrection(nv,column,changeindex,dfc1)
    else:
        logger.info("Cleaning of data not required as it doesnt contain wrong values : "+str(dfc1.iloc[10]["date"]))


    logger.info("The (time)cleaned data ")
    logger.info(dfc1.iloc[10]["date"])
###duhb###
    ##time clean done####
    ###################################
    changeindex = []
    logger.info("Checking and cleaning the ip")
    column = "ip"
    resultbrowser,nv = ipclean(column,dfc1)
    logger.info("the number of valid values : "+str(nv))
    if (resultbrowser):
        logger.info("The  data is sufficient for analysis : "+str(dfc1.iloc[10]["date"]))
    else:
        logger.error("The data is not sufficient for analysis : "+str(dfc1.iloc[10]["date"]))
        logger.error("The data is inconsistent for the date : "+str(dfc1.iloc[10]["date"]))
    ################################################################################################
    logger.info("index of find where wrong values are present :")
    logger.debug(changeindex)
    arraysizeforclean = len(changeindex)
    if(arraysizeforclean>0):
        column = "ip"
        logger.info("cleaning the code : "+str(dfc1.iloc[10]["date"]))
        dfc1 = datacorrection(nv,column,changeindex,dfc1)
    else:
        logger.info("Cleaning of data not required as it doesnt contain wrong values : "+str(dfc1.iloc[10]["date"]))


    logger.info("The (ip)cleaned data ")
    logger.info(dfc1.iloc[10]["date"])
######duhb######
    ##ip clean done####
    ###################
    var3 = ["cik","accession","extention"]
    for x in var3:
        logger.info("checking on : "+str(x))
        status = 0
        status = tofindnonzero(x,dfc1)
        if (status):
            logger.info("analysis possible with : "+str(x))
        else:
            logger.error("analysis not possible with : "+str(x))
    ###################
    final = numberrc(dfc1)
    if(final):
        logger.info("Data clean is complete")
    else:
        logger.error("Data has empty entries")
        logger.error("Check for the missing columns")
    ###################
    return dfc1


In [24]:
#To find the stats of request codes in each day
def codestats(code,ds):
    totalentries = len(ds)
    logger.info("Total number of entries in "+str(totalentries))
    counts = ds["code"].value_counts()
    logger.info("The stats of response codes on : "+str(ds.iloc[10]["date"]))
    logger.info(counts)
    logger.info("The total number of "+str(code)+" : "+str(counts[code]))
    logger.info(" percentage"+str((counts[code]/totalentries)*100)+"%")


In [25]:
def sizeanalysis(ds):
    success = ds['size'][ds['code'] == 200.0].sum()
    totalsize = ds['size'].sum()
    logger.info("Total data size accessed on the date : "+str(ds.iloc[10]["date"]))
    logger.info(totalsize)
    logger.info("successful data size accessed on the date : "+str(ds.iloc[10]["date"]))
    logger.info(success)
    logger.info("percentage success"+str((success/totalsize)*100)+"%")
    logger.info("Average size of the dat being accessed : "+str(int(ds['size'].mean())))


In [26]:
def topfiveip(ds):
    logger.info("To find the top 5 IP addresses making maximum number of requets in the day : "+str(ds.iloc[10]['date']))
    logger.info("grouping data by IP")
    ipds = ds.groupby('ip', as_index=False).agg({"cik": "count"})
    logger.info("sorting in descending order")
    ipds=ipds.sort_values(by=['cik'], ascending=False)
    logger.info("TOP 5 IPs")
    ipdsp = ipds.head(5)
    logger.info(ipdsp)
    logger.info("percentage of requests from top 1 IP :"+str(ipdsp.iloc[0]['ip']))
    logger.info(str((ipdsp.iloc[0]['cik']/len(ds))*100))


In [27]:
def busyhour(tds):
    logger.info("To find the bussiest hour of the day : "+str(tds.iloc[10]['date']))
    tds = tds.groupby('time', as_index=False).agg({"cik": "count"})
    totalrequests = tds['cik'].sum()
    i=0
    sumt = 0
    dicth = {}
    j =1
    for x in tds['cik']:
        sumt = sumt + x
        i=i+1
        if(i == 3600):
            dicth[j] = sumt
            j = j+1
            i = 0
            sumt = 0
    logger.info("The summary of number of requests at all hours")
    logger.info(dicth)
    sum(dicth.values())
    maxv = max(dicth.values())
    logger.info("The bussiest hour : "+str(list(dicth.keys())[list(dicth.values()).index(maxv)])+" and Number of requests : "+str(maxv))
    logger.info("percentage of requests : "+str((maxv/totalrequests)*100)+"%")
    logger.info("Average number of requests per hour : "+str(int(sum(dicth.values())/len(dicth.values()))))
    logger.info("Average percentage : "+str(int(((sum(dicth.values())/len(dicth.values()))/totalrequests)*100))+"%")


In [28]:
def extentionanalysis(ds):
    logger.info("To find the top 10 page type requeted in the day : "+str(ds.iloc[10]['date']))
    logger.info("grouping data by extention")
    epds = ds.groupby('extention', as_index=False).agg({"cik": "count"})
    logger.info("sorting in descending order")
    epds=epds.sort_values(by=['cik'], ascending=False)
    logger.info("TOP 10 extentions")
    epdsp = epds.head(10)
    logger.debug(epdsp)
    logger.info("Number of requests with the extention asked frequesntly:"+str(epdsp.iloc[0]['extention']))
    logger.info("Percent : ")
    logger.info(str((epdsp.iloc[0]['cik']/len(ds))*100))


In [29]:
def summaryanalysis(dfa):
    logger.info("Starting the summary analysis on the cleaned data")
    logger.info("Request Response code analysis")
    logger.info("200 OK percentage : Analysis")
    code = 200
    codestats(code,dfa)
    logger.info("404 Not found percentage : Analysis")
    code = 404
    codestats(code,dfa)
    busyhour(dfa)
    topfiveip(dfa)
    sizeanalysis(dfa)
    extentionanalysis(dfa)
    logger.info("Summary analysis completed")


In [ ]:
logger.info("######################")
logger.info("Generating the URL for every year")
logger.info("######################")
generate_link_list()

logger.info("######################")
logger.info("The year is "+str(year))
logger.info("######################")

logger.info("######################")
path2=createfinalfolder(year)
print(path2)
logger.info("######################")

logger.info("######################")
logger.info("Downloading the CSV and creating Data Frames for the year entered by the year")
logger.info("######################")
zip_csv_on_local(year)



#To read the dataframe
logger.info("######################")
logger.info("Reading the dataframe")
logger.info("######################")

In [73]:
###############################################################################

path2=createfinalfolder(year)
print(path2)

for index in range(0,2):
    path = str(os.getcwd())+'/'+year
    
    df=create_dataframes(index,path)

    logger.info("######################")
    logger.info("The dataframe created successfully")
    logger.info("######################")

    logger.info("######################")
    logger.info("printing the dataframes")
    logger.info("######################")
    logger.info(df.head(5))

    logger.info("######################")
    logger.info("Cleaning the data")
    logger.info("######################")

    #df1 = dataclean(df1)
    #df6 = dataclean(df6)
    df = dataclean(df)

    
    logger.info("######################")
    logger.info("After data clean")
    logger.info("######################")

    
    convert_dataframes_to_csv(df)
    logger.info("######################")
    logger.info("Converting each dataframe to csv")
    logger.info("######################")

    logger.info("######################")
    logger.info("Summary Analysis")
    logger.info("######################")

    #summaryanalysis(df1)
    #summaryanalysis(df6)
    summaryanalysis(df)
    
    
    logger.info("######################")
    logger.info("Zipping the file with CSVs and making a zip folder")
    logger.info("######################")

    zipdirmain()
    logger.info("######################")
    logger.info("DONE")
    logger.info("######################")


C:\Users\DELL\Desktop\ADS\Untitled Folder/201720181012_193929


2018-10-12 19:40:45,614 - INFO - ######################
2018-10-12 19:40:45,614 - INFO - ######################
2018-10-12 19:40:45,614 - INFO - ######################
2018-10-12 19:40:45,614 - INFO - ######################
2018-10-12 19:40:45,614 - INFO - ######################
2018-10-12 19:40:45,618 - INFO - The dataframe created successfully
2018-10-12 19:40:45,618 - INFO - The dataframe created successfully
2018-10-12 19:40:45,618 - INFO - The dataframe created successfully
2018-10-12 19:40:45,618 - INFO - The dataframe created successfully
2018-10-12 19:40:45,618 - INFO - The dataframe created successfully
2018-10-12 19:40:45,629 - INFO - ######################
2018-10-12 19:40:45,629 - INFO - ######################
2018-10-12 19:40:45,629 - INFO - ######################
2018-10-12 19:40:45,629 - INFO - ######################
2018-10-12 19:40:45,629 - INFO - ######################
2018-10-12 19:40:45,634 - INFO - ######################
2018-10-12 19:40:45,634 - INFO - ###########

2018-10-12 19:40:45,834 - INFO - Replacing all the null values with zero
2018-10-12 19:40:45,834 - INFO - Replacing all the null values with zero
2018-10-12 19:41:04,227 - INFO - NaN values are replaced successfuly
2018-10-12 19:41:04,227 - INFO - NaN values are replaced successfuly
2018-10-12 19:41:04,227 - INFO - NaN values are replaced successfuly
2018-10-12 19:41:04,227 - INFO - NaN values are replaced successfuly
2018-10-12 19:41:04,227 - INFO - NaN values are replaced successfuly
2018-10-12 19:41:04,232 - INFO - The null report after cleaning
2018-10-12 19:41:04,232 - INFO - The null report after cleaning
2018-10-12 19:41:04,232 - INFO - The null report after cleaning
2018-10-12 19:41:04,232 - INFO - The null report after cleaning
2018-10-12 19:41:04,232 - INFO - The null report after cleaning
2018-10-12 19:41:04,244 - INFO - Finding the null values
2018-10-12 19:41:04,244 - INFO - Finding the null values
2018-10-12 19:41:04,244 - INFO - Finding the null values
2018-10-12 19:41:0

2018-10-12 19:41:28,681 - INFO - Cleaning of data not required as it doesnt contain wrong values : 2017-01-01
2018-10-12 19:41:28,681 - INFO - Cleaning of data not required as it doesnt contain wrong values : 2017-01-01
2018-10-12 19:41:28,681 - INFO - Cleaning of data not required as it doesnt contain wrong values : 2017-01-01
2018-10-12 19:41:28,681 - INFO - Cleaning of data not required as it doesnt contain wrong values : 2017-01-01
2018-10-12 19:41:28,687 - INFO - The (browser)cleaned data 
2018-10-12 19:41:28,687 - INFO - The (browser)cleaned data 
2018-10-12 19:41:28,687 - INFO - The (browser)cleaned data 
2018-10-12 19:41:28,687 - INFO - The (browser)cleaned data 
2018-10-12 19:41:28,687 - INFO - The (browser)cleaned data 
2018-10-12 19:41:28,705 - INFO - 2017-01-01
2018-10-12 19:41:28,705 - INFO - 2017-01-01
2018-10-12 19:41:28,705 - INFO - 2017-01-01
2018-10-12 19:41:28,705 - INFO - 2017-01-01
2018-10-12 19:41:28,705 - INFO - 2017-01-01
2018-10-12 19:41:28,709 - INFO - Checkin

2018-10-12 19:42:42,063 - INFO - cleaning the column : crawler
2018-10-12 19:42:42,063 - INFO - cleaning the column : crawler
2018-10-12 19:42:42,063 - INFO - cleaning the column : crawler
2018-10-12 19:42:42,063 - INFO - cleaning the column : crawler
2018-10-12 19:42:42,068 - INFO - The total entries in the dataframe : 17485535
2018-10-12 19:42:42,068 - INFO - The total entries in the dataframe : 17485535
2018-10-12 19:42:42,068 - INFO - The total entries in the dataframe : 17485535
2018-10-12 19:42:42,068 - INFO - The total entries in the dataframe : 17485535
2018-10-12 19:42:42,068 - INFO - The total entries in the dataframe : 17485535
2018-10-12 19:42:42,075 - INFO - The valid values : 
2018-10-12 19:42:42,075 - INFO - The valid values : 
2018-10-12 19:42:42,075 - INFO - The valid values : 
2018-10-12 19:42:42,075 - INFO - The valid values : 
2018-10-12 19:42:42,075 - INFO - The valid values : 
2018-10-12 19:42:42,080 - INFO - ['0.0', '1.0']
2018-10-12 19:42:42,080 - INFO - ['0.0',

...

2018-10-12 19:52:50,227 - INFO - ######################
2018-10-12 19:52:50,227 - INFO - ######################
2018-10-12 19:52:50,227 - INFO - ######################
2018-10-12 19:52:50,227 - INFO - ######################
2018-10-12 19:52:50,227 - INFO - ######################
2018-10-12 19:52:50,233 - INFO - Converting each dataframe to csv
2018-10-12 19:52:50,233 - INFO - Converting each dataframe to csv
2018-10-12 19:52:50,233 - INFO - Converting each dataframe to csv
2018-10-12 19:52:50,233 - INFO - Converting each dataframe to csv
2018-10-12 19:52:50,233 - INFO - Converting each dataframe to csv
2018-10-12 19:52:50,239 - INFO - ######################
2018-10-12 19:52:50,239 - INFO - ######################
2018-10-12 19:52:50,239 - INFO - ######################
2018-10-12 19:52:50,239 - INFO - ######################
2018-10-12 19:52:50,239 - INFO - ######################
2018-10-12 19:52:50,244 - INFO - ######################
2018-10-12 19:52:50,244 - INFO - #####################

2018-10-12 19:52:51,340 - INFO - The total number of 404 : 93691
2018-10-12 19:52:51,340 - INFO - The total number of 404 : 93691
2018-10-12 19:52:51,340 - INFO - The total number of 404 : 93691
2018-10-12 19:52:51,340 - INFO - The total number of 404 : 93691
2018-10-12 19:52:51,340 - INFO - The total number of 404 : 93691
2018-10-12 19:52:51,346 - INFO -  percentage0.535820036390079%
2018-10-12 19:52:51,346 - INFO -  percentage0.535820036390079%
2018-10-12 19:52:51,346 - INFO -  percentage0.535820036390079%
2018-10-12 19:52:51,346 - INFO -  percentage0.535820036390079%
2018-10-12 19:52:51,346 - INFO -  percentage0.535820036390079%
2018-10-12 19:52:51,351 - INFO - To find the bussiest hour of the day : 2017-01-01
2018-10-12 19:52:51,351 - INFO - To find the bussiest hour of the day : 2017-01-01
2018-10-12 19:52:51,351 - INFO - To find the bussiest hour of the day : 2017-01-01
2018-10-12 19:52:51,351 - INFO - To find the bussiest hour of the day : 2017-01-01
2018-10-12 19:52:51,351 - IN

2018-10-12 19:52:56,083 - INFO - 2523401101916.0
2018-10-12 19:52:56,083 - INFO - 2523401101916.0
2018-10-12 19:52:56,083 - INFO - 2523401101916.0
2018-10-12 19:52:56,083 - INFO - 2523401101916.0
2018-10-12 19:52:56,089 - INFO - successful data size accessed on the date : 2017-01-01
2018-10-12 19:52:56,089 - INFO - successful data size accessed on the date : 2017-01-01
2018-10-12 19:52:56,089 - INFO - successful data size accessed on the date : 2017-01-01
2018-10-12 19:52:56,089 - INFO - successful data size accessed on the date : 2017-01-01
2018-10-12 19:52:56,089 - INFO - successful data size accessed on the date : 2017-01-01
2018-10-12 19:52:56,095 - INFO - 2490017985297.0
2018-10-12 19:52:56,095 - INFO - 2490017985297.0
2018-10-12 19:52:56,095 - INFO - 2490017985297.0
2018-10-12 19:52:56,095 - INFO - 2490017985297.0
2018-10-12 19:52:56,095 - INFO - 2490017985297.0
2018-10-12 19:52:56,110 - INFO - percentage success98.6770586493898%
2018-10-12 19:52:56,110 - INFO - percentage succes

2018-10-12 19:54:17,680 - INFO - DONE
2018-10-12 19:54:17,680 - INFO - DONE
2018-10-12 19:54:17,680 - INFO - DONE
2018-10-12 19:54:17,684 - INFO - ######################
2018-10-12 19:54:17,684 - INFO - ######################
2018-10-12 19:54:17,684 - INFO - ######################
2018-10-12 19:54:17,684 - INFO - ######################
2018-10-12 19:54:17,684 - INFO - ######################


ParserError: Error tokenizing data. C error: out of memory

In [36]:
createfinalfolder(year)

20181012_163650
C:\Users\DELL\Desktop\ADS\Untitled Folder/201620181012_163650
enter2017


In [53]:
df.head()

,ip,date,time,zone,cik,accession,extention,code,size,idx,norefer,noagent,find,crawler,browser
0,104.197.32.ihd,2017-01-01,00:00:00,0.0,1111711.0,0001193125-12-324016,-index.htm,200.0,7627.0,1.0,0.0,0.0,10.0,0.0,0
1,104.197.32.ihd,2017-01-01,00:00:00,0.0,720005.0,0000720005-16-000144,-index.htm,200.0,12014.0,1.0,0.0,0.0,10.0,0.0,0
2,104.197.32.ihd,2017-01-01,00:00:00,0.0,1111711.0,0001193125-12-386591,-index.htm,200.0,6581.0,1.0,0.0,0.0,10.0,0.0,0
3,104.197.32.ihd,2017-01-01,00:00:00,0.0,720005.0,0000720005-16-000121,-index.htm,200.0,11255.0,1.0,0.0,0.0,10.0,0.0,0
4,107.178.195.aea,2017-01-01,00:00:00,0.0,799288.0,0000799288-16-000197,le-20161028_cal.xml,301.0,624.0,0.0,0.0,0.0,10.0,0.0,0


In [58]:
def convert_dataframes_to_csv(df):
    ts = time.time()
    dt = datetime.datetime.fromtimestamp(ts).strftime('%Y%m%d_%H%M%S')
    #folder=str(dt)
    df = pd.DataFrame(df)
    #csvfilename = str(year)+"-01"
    csvfilename=str(dt)
    print(path2)
    df.to_csv(path2+'/'+csvfilename+'.csv')

In [59]:
convert_dataframes_to_csv(df)

C:\Users\DELL\Desktop\ADS\Untitled Folder/201720181012_174348


In [63]:
def zipdir(path, ziph):
    path=str(path)
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))



In [64]:
if __name__ == '__main__':
    zipf = zipfile.ZipFile('Python.zip', 'w', zipfile.ZIP_DEFLATED)
    zipdir(path2, zipf)
    zipf.close()

In [66]:
def zipdirmain():
    zipf = zipfile.ZipFile('Python.zip', 'w', zipfile.ZIP_DEFLATED)
    zipdir(path2, zipf)
    zipf.close()

In [68]:
zipdirmain()